## Imports and Setup
Updated with NOISE_STD, VGG_WEIGHT, and library imports. Checks for GPU availability

In [ ]:
import os
import json
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, random_split
# FIXED: Updated AMP imports
from torch.amp import autocast, GradScaler 
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm  # FIXED: Progress bar
import matplotlib.pyplot as plt

# Set seeds
torch.manual_seed(42)
np.random.seed(42)

# --- Configuration ---
BATCH_SIZE = 4
LEARNING_RATE = 1e-4
EPOCHS = 100
PHYSICS_WEIGHT = 0.1
VGG_WEIGHT = 0.1
OUTPUT_DIR = "/kaggle/working"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")

# Dataset Discovery
search_path = "/kaggle/input/**/*.pt"
pt_files = glob.glob(search_path, recursive=True)
if not pt_files:
    raise FileNotFoundError("❌ Dataset not found!")
DATA_FILE = pt_files[0]
DATA_DIR = os.path.dirname(DATA_FILE)
STATS_FILE = os.path.join(DATA_DIR, "normalization_stats.json")

## Dataset Class 
Added ```noise_std``` parameter for Sim-to-Real gap augmentation.

In [ ]:
class FluidLoader(Dataset):
    def __init__(self, pt_file, stats_file, target_res=256):
        print(f"⏳ Loading data map...")
        try:
            data = torch.load(pt_file, map_location='cpu', mmap=True)
        except:
            data = torch.load(pt_file, map_location='cpu')
            
        self.inputs = data['inputs']
        self.targets = data['targets']
        self.target_res = target_res
        
        # FIXED: Check resolution once at startup
        # We check the first sample to determine if upscaling is needed
        sample_h = self.inputs.shape[-1]
        self.needs_upscale = (sample_h != target_res)
        
        # Load K
        if 'K' in data:
            self.K = float(data['K'])
        elif os.path.exists(stats_file):
            with open(stats_file, 'r') as f:
                stats = json.load(f)
            self.K = float(stats['scaling_factor'])
        else:
            self.K = 1.0
            
        print(f"✅ Loaded {len(self.inputs)} samples. Upscaling: {self.needs_upscale}")

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        # FIXED: Removed .clone() (F.interpolate creates a new tensor anyway)
        lr = self.inputs[idx] 
        hr = self.targets[idx]
        
        # FIXED: Use pre-calculated flag instead of checking shape every time
        if self.needs_upscale:
            lr = F.interpolate(
                lr.unsqueeze(0), 
                size=(self.target_res, self.target_res), 
                mode='bilinear', 
                align_corners=False
            ).squeeze(0)
            
        return lr, hr

## Model Architecture (ResUNet)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(residual)
        out = self.relu(out)
        return out

class ResUNet(nn.Module):
    def __init__(self, in_channels=6, out_channels=2, features=[64, 128, 256, 512]):
        super(ResUNet, self).__init__()
        self.encoder = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Encoder
        self.input_conv = nn.Sequential(
            nn.Conv2d(in_channels, features[0], kernel_size=3, padding=1),
            nn.BatchNorm2d(features[0]),
            nn.ReLU(inplace=True)
        )
        input_feat = features[0]
        for feature in features:
            self.encoder.append(ResidualBlock(input_feat, feature))
            input_feat = feature

        # Bottleneck
        self.bottleneck = ResidualBlock(features[-1], features[-1] * 2)

        # Decoder
        self.upconvs = nn.ModuleList()
        self.decoder = nn.ModuleList()
        features = features[::-1]
        for feature in features:
            self.upconvs.append(nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2))
            self.decoder.append(ResidualBlock(feature * 2, feature))

        # Output
        self.final_conv = nn.Conv2d(features[-1], out_channels, kernel_size=1)
        
    def forward(self, x):
        skip_connections = []
        out = self.input_conv(x)
        
        for layer in self.encoder:
            out = layer(out)
            skip_connections.append(out)
            out = self.pool(out)
            
        out = self.bottleneck(out)
        skip_connections = skip_connections[::-1] 
        
        for idx in range(len(self.decoder)):
            out = self.upconvs[idx](out)
            skip = skip_connections[idx]
            if out.shape != skip.shape:
                out = F.interpolate(out, size=skip.shape[2:], mode='bilinear', align_corners=False)
            concat_skip = torch.cat((skip, out), dim=1)
            out = self.decoder[idx](concat_skip)
            
        out = self.final_conv(out)
        
        return out

## Loss Functions ( VGG, Physics Informed)

In [ ]:
class VGGLoss(nn.Module):
    def __init__(self):
        super(VGGLoss, self).__init__()
        vgg = models.vgg19(weights=models.VGG19_Weights.DEFAULT)
        self.feature_extractor = nn.Sequential(*list(vgg.features.children())[:35]).eval()
        
        for param in self.feature_extractor.parameters():
            param.requires_grad = False
            
        # FIXED: ImageNet Normalization constants as buffers
        self.register_buffer("mean", torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        self.register_buffer("std", torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))

    def forward(self, input, target):
        # 1. Expand 2 channel -> 3 channel (Append Zero channel)
        # Using pad is more efficient than creating zeros and catting
        # Pad format: (left, right, top, bottom, front, back) for last 3 dims? 
        # F.pad for 4D input (B, C, H, W) pads starting from last dim.
        # We want to pad Channel dim. F.pad tuple is (W_l, W_r, H_t, H_b, C_f, C_b)?? No.
        # Simple concat is safer for clarity, but lets optimize allocation.
        
        b, c, h, w = input.shape
        # Create zero channel ONCE per batch if size constant? 
        # Just stick to cat for safety but ensure device match.
        zeros = torch.zeros(b, 1, h, w, device=input.device, dtype=input.dtype)
        
        input_3c = torch.cat([input, zeros], dim=1)
        target_3c = torch.cat([target, zeros], dim=1)
        
        # 2. FIXED: Normalize to ImageNet stats
        # VGG expects [0, 1] usually, but our data is normalized by K.
        # We assume input is roughly in reasonable range, but centering helps.
        input_norm = (input_3c - self.mean) / self.std
        target_norm = (target_3c - self.mean) / self.std
        
        input_features = self.feature_extractor(input_norm)
        target_features = self.feature_extractor(target_norm)
        return F.mse_loss(input_features, target_features)

class DivergenceLoss(nn.Module):
    def __init__(self):
        super(DivergenceLoss, self).__init__()

    def forward(self, output, scaling_factor):
        u = output[:, 0, :, :] * scaling_factor
        v = output[:, 1, :, :] * scaling_factor
        du_dx = u[:, :, 1:] - u[:, :, :-1]
        dv_dy = v[:, 1:, :] - v[:, :-1, :]
        return torch.mean((du_dx[:, :-1, :] + dv_dy[:, :, :-1])**2)

## Training Loop

In [ ]:
def train(use_physics=False, mse_weight=1.0, vgg_weight=0.1, physics_weight=0.1):
    """
    Args:
        use_physics (bool): Whether to enable divergence loss.
        mse_weight (float): Weight for Pixel Loss (MSE).
        vgg_weight (float): Weight for Perceptual Loss (VGG).
        physics_weight (float): Weight for Divergence Loss (if use_physics=True).
    """
    # 1. Data
    dataset = FluidLoader(DATA_FILE, STATS_FILE)
    train_sz = int(0.8 * len(dataset))
    val_sz = len(dataset) - train_sz
    train_ds, val_ds = random_split(dataset, [train_sz, val_sz], generator=torch.Generator().manual_seed(42))
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    # 2. Setup
    model = ResUNet(in_channels=6, out_channels=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    scaler = GradScaler('cuda') 
    
    # Loss Functions
    mse_fn = nn.MSELoss()
    vgg_fn = VGGLoss().to(device)
    div_fn = DivergenceLoss().to(device)
    K = dataset.K
    
    mode_name = "PINN" if use_physics else "Baseline"
    print(f"\n🚀 Starting {mode_name} Training...")
    print(f"   > Weights: MSE={mse_weight} | VGG={vgg_weight} | Physics={physics_weight if use_physics else 0.0}")
    
    hist = {'train': [], 'val': [], 'lr': []}
    best_val_loss = float('inf')
    patience = 7
    counter = 0

    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0.0
        
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False)
        
        for x, y in loop:
            x, y = x.to(device), y.to(device)
            
            optimizer.zero_grad(set_to_none=True) 
            
            with autocast('cuda'): 
                pred = model(x)
                
                # --- Weighted Loss Calculation ---
                loss = 0.0
                
                # 1. MSE Loss (Pixel Accuracy)
                if mse_weight > 0:
                    loss += mse_weight * mse_fn(pred, y)
                
                # 2. VGG Loss (Texture/Sharpness)
                if vgg_weight > 0:
                    loss += vgg_weight * vgg_fn(pred, y)
                
                # 3. Physics Loss (Divergence/Mass)
                if use_physics and physics_weight > 0:
                    loss += physics_weight * div_fn(pred, K)
            
            scaler.scale(loss).backward()
            
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            loop.set_postfix(loss=loss.item())
            
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                with autocast('cuda'):
                    pred = model(x)
                    # Track MSE for early stopping (standard metric)
                    val_loss += mse_fn(pred, y).item()
        
        avg_train = train_loss / len(train_loader)
        avg_val = val_loss / len(val_loader)
        
        # Step Scheduler
        scheduler.step(avg_val)
        current_lr = optimizer.param_groups[0]['lr']
        
        hist['train'].append(avg_train)
        hist['val'].append(avg_val)
        hist['lr'].append(current_lr)
        
        print(f"Epoch {epoch+1} | Train: {avg_train:.6f} | Val: {avg_val:.6f} | LR: {current_lr:.2e}")
        
        # Checkpointing
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_val,
        }
        
        if avg_val < best_val_loss:
            best_val_loss = avg_val
            counter = 0
            torch.save(checkpoint, f"{OUTPUT_DIR}/{mode_name}_best.pth")
            print("  --> New Best Model Saved!")
        else:
            counter += 1
            if counter >= patience:
                print("🛑 Early Stopping triggered.")
                break
                
    # Plotting
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(hist['train'], label='Train')
    plt.plot(hist['val'], label='Val')
    plt.title(f'{mode_name} Loss')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(hist['lr'], color='orange')
    plt.title('Learning Rate')
    plt.grid(True, alpha=0.3)
    
    plt.savefig(f"{OUTPUT_DIR}/{mode_name}_history.png")
    plt.show()
    
    best_checkpoint = torch.load(f"{OUTPUT_DIR}/{mode_name}_best.pth")
    model.load_state_dict(best_checkpoint['model_state_dict'])
    
    return model, dataset

## Execute
Run this cell to start.

In [ ]:
# Configuration 1: Standard (Balanced)
model, ds = train(
    use_physics=True, 
    mse_weight=1.0, 
    vgg_weight=0.1, 
    physics_weight=0.1
)

# Configuration 2: High Sharpness (Native Data Fine-Tuning)
# model, ds = train(
#     use_physics=True, 
#     mse_weight=0.05,   # Low MSE to allow spatial shift
#     vgg_weight=1.0,    # High VGG for texture
#     physics_weight=0.2 # Physics to keep it valid
# )